In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


**Import Libriries**

In [3]:
import numpy as np 
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

**Loading Datasets**

In [4]:
d_item_cat = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv")
d_item_cat.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [5]:
d_items = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/items.csv")
d_items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [6]:
d_sam_sub = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv")
d_sam_sub.head()

,ID,item_cnt_month
0,0,0.5
1,1,0.5
2,2,0.5
3,3,0.5
4,4,0.5


In [7]:
d_sal_tra = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv") 
d_sal_tra.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [8]:
d_shop = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/shops.csv")
d_shop.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [9]:
d_test = pd.read_csv("/kaggle/input/competitive-data-science-predict-future-sales/test.csv")
d_test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


In [47]:
dt_test = d_test.drop(labels = "ID",axis = 1)
dt_test

,shop_id,item_id
0,5,5037
1,5,5320
2,5,5233
3,5,5232
4,5,5268
...,...,...
214195,45,18454
214196,45,16188
214197,45,15757
214198,45,19648


**Check The Null Values** 

In [10]:
print("items categorical null values",d_item_cat.isnull().sum().sum())
print("items  null values",d_items.isnull().sum().sum())
print("sales train null values",d_sal_tra.isnull().sum().sum())
print("test data null values",d_test.isnull().sum().sum())

items categorical null values 0
items  null values 0
sales train null values 0
test data null values 0


**Check The Unique Values**

In [11]:
print("items categorical unique values",d_item_cat.nunique())
print("items  unique values",d_items.nunique())
print("sales train unique values",d_sal_tra.nunique())
print("test data unique values",d_test.nunique())

items categorical unique values item_category_name    84
item_category_id      84
dtype: int64
items  unique values item_name           22170
item_id             22170
item_category_id       84
dtype: int64
sales train unique values date               1034
date_block_num       34
shop_id              60
item_id           21807
item_price        19993
item_cnt_day        198
dtype: int64
test data unique values ID         214200
shop_id        42
item_id      5100
dtype: int64


**Mearge The Data**

In [12]:
new_data = d_sal_tra.copy()
new_data

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0
...,...,...,...,...,...,...
2935844,10.10.2015,33,25,7409,299.00,1.0
2935845,09.10.2015,33,25,7460,299.00,1.0
2935846,14.10.2015,33,25,7459,349.00,1.0
2935847,22.10.2015,33,25,7440,299.00,1.0


In [13]:
new_data = pd.merge(d_item_cat,d_items,how = "inner",on="item_category_id")
new_data

,item_category_name,item_category_id,item_name,item_id
0,PC - Гарнитуры/Наушники,0,PC: Гарнитура HyperX Cloud Core игровая стерео...,5441
1,PC - Гарнитуры/Наушники,0,Наушники PHILIPS SBC HC8680,16255
2,PC - Гарнитуры/Наушники,0,Наушники RITMIX RH-120,16256
3,PC - Гарнитуры/Наушники,0,Наушники RITMIX RH-124 Black,16257
4,Аксессуары - PS2,1,PS2: Карта памяти 8 Мб черная (Memory Card 8Mb...,5606
...,...,...,...,...
22165,Элементы питания,83,Элемент питания GP 15AU (LR6) - Блистер 2 шт U...,22095
22166,Элементы питания,83,Элемент питания GP 24A (LR03) - Блистер 2 шт,22096
22167,Элементы питания,83,Элемент питания GP 24AU (LR03) - Блистер 2 шт ...,22097
22168,Элементы питания,83,Элемент питания КОСМОС LR03 2*BL,22098


In [14]:
new_data = pd.merge(new_data,d_sal_tra,how = "inner",on="item_id")
new_data

,item_category_name,item_category_id,item_name,item_id,date,date_block_num,shop_id,item_price,item_cnt_day
0,PC - Гарнитуры/Наушники,0,Наушники PHILIPS SBC HC8680,16255,28.02.2013,1,0,93.0,1.0
1,PC - Гарнитуры/Наушники,0,Наушники RITMIX RH-120,16256,02.04.2013,3,38,22.4,1.0
2,PC - Гарнитуры/Наушники,0,Наушники RITMIX RH-124 Black,16257,10.01.2013,0,25,148.0,1.0
3,Аксессуары - PS2,1,PS2: Карта памяти 8 Мб черная (Memory Card 8Mb...,5606,10.01.2013,0,45,148.0,1.0
4,Аксессуары - PS2,1,Pelican. Геймпад Nerf Wireless Controller PS2 ...,5740,20.02.2013,1,0,283.0,1.0
...,...,...,...,...,...,...,...,...,...
2935844,Элементы питания,83,Элемент питания GP 24AU (LR03) - Блистер 2 шт ...,22097,12.07.2013,6,10,37.0,1.0
2935845,Элементы питания,83,Элемент питания КОСМОС LR03 2*BL,22098,11.12.2013,11,10,7.0,1.0
2935846,Элементы питания,83,Элемент питания СТАРТ ZT 15A (LR6) - Блистер 2 шт,22099,19.07.2013,6,2,40.0,1.0
2935847,Элементы питания,83,Элемент питания СТАРТ ZT 15A (LR6) - Блистер 2 шт,22099,20.11.2013,10,16,40.0,1.0


In [15]:
new_data = pd.merge(new_data,d_shop,how = "inner",on="shop_id")
new_data

,item_category_name,item_category_id,item_name,item_id,date,date_block_num,shop_id,item_price,item_cnt_day,shop_name
0,PC - Гарнитуры/Наушники,0,Наушники PHILIPS SBC HC8680,16255,28.02.2013,1,0,93.00,1.0,"!Якутск Орджоникидзе, 56 фран"
1,Аксессуары - PS2,1,Pelican. Геймпад Nerf Wireless Controller PS2 ...,5740,20.02.2013,1,0,283.00,1.0,"!Якутск Орджоникидзе, 56 фран"
2,Аксессуары - PS3,2,PS Move Controller (Контроллер движений),5570,28.02.2013,1,0,93.00,1.0,"!Якутск Орджоникидзе, 56 фран"
3,Аксессуары - PS3,2,PS Move Motion Controller (Контроллер движений...,5572,04.01.2013,0,0,1322.00,3.0,"!Якутск Орджоникидзе, 56 фран"
4,Аксессуары - PS3,2,PS Move Motion Controller (Контроллер движений...,5572,05.01.2013,0,0,1322.00,2.0,"!Якутск Орджоникидзе, 56 фран"
...,...,...,...,...,...,...,...,...,...,...
2935844,Подарки - Фигурки,72,Фигурка Wolverine Marvel Classic Wolverine Hea...,20886,03.10.2014,21,20,1500.68,1.0,"Москва ""Распродажа"""
2935845,Подарки - Фигурки,72,Фигурка Wolverine Marvel Classic Wolverine Hea...,20886,04.10.2014,21,20,1499.00,3.0,"Москва ""Распродажа"""
2935846,Подарки - Фигурки,72,Фигурка Wolverine Marvel Classic Wolverine Hea...,20886,05.10.2014,21,20,1499.00,6.0,"Москва ""Распродажа"""
2935847,Элементы питания,83,Элемент питания DURACELL LR06-BC2,22088,01.10.2015,33,20,119.00,1.0,"Москва ""Распродажа"""


In [21]:
dx = ["shop_id","item_id"]
x = new_data[dx]
y = new_data["item_cnt_day"]
x,y

(         shop_id  item_id
 0              0    16255
 1              0     5740
 2              0     5570
 3              0     5572
 4              0     5572
 ...          ...      ...
 2935844       20    20886
 2935845       20    20886
 2935846       20    20886
 2935847       20    22088
 2935848       20    22088
 
 [2935849 rows x 2 columns],
 0          1.0
 1          1.0
 2          1.0
 3          3.0
 4          2.0
           ... 
 2935844    1.0
 2935845    3.0
 2935846    6.0
 2935847    1.0
 2935848    2.0
 Name: item_cnt_day, Length: 2935849, dtype: float64)

**LinearRegression Model**

In [ ]:
from sklearn.model_selection import train_test_split

In [31]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size = 0.3,random_state = 20)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((2055094, 2), (880755, 2), (2055094,), (880755,))

In [24]:
from sklearn.linear_model import LinearRegression

In [25]:
lr = LinearRegression()

In [26]:
lr.fit(x_train,y_train)

LinearRegression()

In [32]:
pred = lr.predict(x_test)
pred

array([1.27026309, 1.29404548, 1.25485036, ..., 1.31072014, 1.21822287,
       1.2413382 ])

In [33]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [34]:
r2_score(y_test,pred)

0.0002462216265555206

In [35]:
r2_score(y_train,lr.predict(x_train))

0.00035827763633455856

In [48]:
test_data = lr.predict(dt_test)

In [49]:
test_data

array([1.23018628, 1.23209938, 1.23151125, ..., 1.26982918, 1.29613261,
       1.16986128])

**Output**

In [51]:
sub = pd.DataFrame({'ID':d_test.ID.values, 'item_cnt_month':test_data})
sub

,ID,item_cnt_month
0,0,1.230186
1,1,1.232099
2,2,1.231511
3,3,1.231504
4,4,1.231748
...,...,...
214195,214195,1.288061
214196,214196,1.272743
214197,214197,1.269829
214198,214198,1.296133


**XGBRegressor Model**

In [54]:
from xgboost import XGBRegressor
xgb = XGBRegressor(subsample= 1,max_depth=5,learning_rate=0.05,gamma=1,reg_alpha=10,n_estimators=150,colsample_bylevel=1)

In [55]:
xgb.fit(x_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=1, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=150, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=10, reg_lambda=1, ...)

In [57]:
pred1 = xgb.predict(x_test)
pred1

array([1.0997641, 1.3817836, 1.0910659, ..., 1.1039033, 1.137784 ,
       1.1551332], dtype=float32)

In [58]:
r2_score(y_test,pred1)

0.11118755025637483

In [59]:
r2_score(y_train,xgb.predict(x_train))

0.1649255581668897

In [62]:
test_data1 = xgb.predict(dt_test)
test_data1

array([1.0867535, 1.0867535, 1.0867535, ..., 1.0708563, 1.0612885,
       1.0259292], dtype=float32)

In [63]:
sub1 = pd.DataFrame({'ID':d_test.ID.values, 'item_cnt_month':test_data1})
sub1

,ID,item_cnt_month
0,0,1.086753
1,1,1.086753
2,2,1.086753
3,3,1.086753
4,4,1.086753
...,...,...
214195,214195,1.037218
214196,214196,1.079832
214197,214197,1.070856
214198,214198,1.061288
